In [1]:
import numpy as np
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import random
import  scipy.io
from scipy import signal
import cv2
import keras
from keras.datasets import cifar10
import torch
from torch.nn.functional import conv2d, conv3d

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [0]:
num_classes = 10
epochs = 100
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
def plot_filters(filters):
    fig, axes = plt.subplots(6, 6, figsize=(15,5))
    itr = 0
    for i in range(6):
        for j in range(6):
            axes[i, j].imshow(filters[:, :,itr])
            axes[i, j].set_xticks([])
            axes[i, j].set_yticks([])
            itr += 1

In [0]:
kernal1=np.random.randn(5,5,18) #1st Conv
kernal2=np.random.randn(3,3,18) #second Conv

In [0]:
def size_for_convolution(image_dim,kernal_dim,padding,stride):
    height = int((image_dim[0]-kernal_dim[0]+2*padding)/stride)+1
    width = int((image_dim[1]-kernal_dim[1]+2*padding)/stride)+1
    return height,width
def conv_single_step(img_patch, filtr):
    filtr=np.expand_dims(filtr,axis=-1)
    conv=np.multiply(img_patch,filtr)
    return np.sum(conv)
def conv_forward(sample, filters, stride=1, pad=0):
    height, width=size_for_convolution(sample.shape,filters.shape,pad,stride)
    result=np.zeros((height,width,filters.shape[2]))
    for f in range(0,filters.shape[2]):
        filtr=filters[:,:,f]
        i=j=0
        filter_width=filter_height=filters.shape[0]
        while i<height:
            j=0
            while j<width:
                patch=sample[i:i+filter_height,j:j+filter_width]
                output=conv_single_step(patch,filtr)
                result[i,j,f]=output
                #print(result.shape)
                j=j+stride
            i=i+stride        
    return result

In [0]:
def convolution(a,b):
    a=np.expand_dims(a,axis=-1)
    a=np.expand_dims(a,axis=-1)
    a=a.T
    b=b.T
    b=np.expand_dims(b,axis=1)
    inputs = torch.from_numpy(a).float()
    filters = torch.from_numpy(b).float()
    out = torch.nn.functional.conv2d(inputs, filters,stride=1,padding=0, bias=None)
    out=np.squeeze(np.array(out),axis=0)
    return out.T

In [0]:
def Maxpool(conv_result):
    maxID=np.zeros((conv_result.shape[0],conv_result.shape[1],conv_result.shape[2]))
    stride = 2
    fs = 2
    a=conv_result
    conv_mats = np.array([ [ [ [np.max(a[j:j+fs,k:k+fs,i]),([j,k]+np.array(np.unravel_index(np.argmax(a[j:j+fs,k:k+fs,i] , axis=None),a[j:j+fs,k:k+fs,i].shape)))] for k in range(0,a.shape[1]-fs+1,stride)] for j in range(0,a.shape[0]-fs+1,stride)] for i in range(0,a.shape[-1])])
    #conv_mats=conv_mats.reshape(conv_mats.shape[0]*conv_mats.shape[1]*conv_mats.shape[2],2)
    return conv_mats
def Mask(index,conv_res):
    maxID=np.zeros((conv_res.shape[0],conv_res.shape[1],conv_res.shape[2]))
    #print(index.shape)
    i=j=k=0 
    # the range of i must be from 0 to 10.
    # the range of j must be from 0 to 10.
    # the range of k must be from 0 to 35.
    while(k<index.shape[0]):
        m=index[k][i][j][1][0]
        n=index[k][i][j][1][1]
        maxID[m][n][k] = 1
        j += 1
        if(j%index.shape[1]==0):
            j=0
            i+= 1
            if (i%index.shape[2]==0):
                i=0
                k+= 1
    return maxID
def MeanImageSubtraction(all_data):
    m_data=np.mean(all_data,axis=0)
    m_data=np.expand_dims(m_data,axis=-1)
    return m_data

In [0]:
def sigmoid(s):
    return 1/(1+np.exp(-s))
def sigmoid_derivative(s):
    return s*(1-s)
def MeanImageSubtraction(all_data):
    m_data=np.mean(all_data,axis=0)
    m_data=np.expand_dims(m_data,axis=-1)
    return m_data
def softmax(s):
    s=np.exp(s)
    sum_s=np.sum(s)
    #sum_s=np.expand_dims(sum_s,axis=1)
    return s/sum_s 
def one_hot_encoding(data):
    Y=[]
    for i in range(0,len(data)):
        temp=np.zeros(10,dtype=int)
        if data[i]==0:
            temp[0]=1
            Y.append(temp)
        if data[i]==1:
            temp[1]=1
            Y.append(temp)
        if data[i]==2:
            temp[2]=1
            Y.append(temp)
        if data[i]==3:
            temp[3]=1
            Y.append(temp)
        if data[i]==4:
            temp[4]=1
            Y.append(temp)
        if data[i]==5:
            temp[5]=1
            Y.append(temp)
        if data[i]==6:
            temp[6]=1
            Y.append(temp)
        if data[i]==7:
            temp[7]=1
            Y.append(temp)
        if data[i]==8:
            temp[8]=1
            Y.append(temp)
        if data[i]==9:
            temp[9]=1
            Y.append(temp)
        #print(temp)
    return np.array(Y)

def crossentropyloss(y_pred,Y):
    cel=0;
    for i in range(0,len(Y)):
        cel+= Y[i]*np.log(y_pred[i])
    return -cel


In [0]:
def init_network(no_of_layers, input_dim,neurons_per_layer):
    layers_params={}

    for i in range(0,len(neurons_per_layer)):
        if i==0:
            layers_params["W1"]=np.random.randn(input_dim,neurons_per_layer[0]) 
            print("W1 Shape",layers_params["W1"].shape)
            layers_params["B1"]=np.random.randn(neurons_per_layer[0],1)
            #layers_params["B1"]=np.expand_dims(layers_params["B1"],axis=-1)
            print("B1 Shape",layers_params["B1"].shape)
            layers_params["Z1"]=np.zeros((neurons_per_layer[0],1))
            #layers_params["Z1"]=np.expand_dims(layers_params["Z1"],axis=-1)
            print("Z1 Shape",layers_params["Z1"].shape)
            layers_params["A1"]=np.zeros((neurons_per_layer[0],1))
            #layers_params["A1"]=np.expand_dims(layers_params["A1"],axis=-1)
            print("A1 Shape",layers_params["A1"].shape)
        else:
            layers_params[f"W{(i+1)}"]=np.random.randn(neurons_per_layer[i-1],neurons_per_layer[i])
            print("W"+str(i+1)+" Shape",layers_params[f"W{(i+1)}"].shape)
            layers_params[f"B{(i+1)}"]=np.random.randn(neurons_per_layer[i],1)
            #layers_params[f"B{(i+1)}"]=np.expand_dims(layers_params[f"B{(i+1)}"],axis=-1)
            print("B"+str(i+1)+" Shape",layers_params[f"B{(i+1)}"].shape)
            layers_params[f"Z{(i+1)}"]=np.zeros((neurons_per_layer[i],1))
            #layers_params[f"Z{(i+1)}"]=np.expand_dims(layers_params[f"Z{(i+1)}"],axis=-1)
            print("Z"+str(i+1)+" Shape",layers_params[f"Z{(i+1)}"].shape)
            layers_params[f"A{(i+1)}"]=np.zeros((neurons_per_layer[i],1))
            #layers_params[f"A{(i+1)}"]=np.expand_dims(layers_params[f"A{(i+1)}"],axis=-1)
            print("A"+str(i+1)+" Shape",layers_params[f"A{(i+1)}"].shape)
    return layers_params

In [0]:
def feedforward(net,trainX, no_of_layers):
    net["Z1"]=np.dot(net["W1"].T,trainX) + net["B1"]
    net["A1"]=sigmoid(net["Z1"])
    for i in  range(1,no_of_layers+1):
        net[f"Z{(i+1)}"]=np.dot(net[f"W{(i+1)}"].T,net[f"A{(i)}"]) +net[f"B{(i+1)}"]
        if i<no_of_layers:
            net[f"A{(i+1)}"]=sigmoid(net[f"Z{(i+1)}"])
            continue
        net[f"A{(i+1)}"]=softmax(net[f"Z{(i+1)}"])
    return net[f"A{(i+1)}"]
def Backwardpropagate(sample, y_pred,Y,net,lr):
    dL_params={}
    ###for last layer 
    Y=np.expand_dims(Y,axis=-1)
    error=y_pred-Y ##error
    dL_params[f"dW{(no_of_layers+1)}"]=np.dot(net["A"+str(no_of_layers)],error.T)
    dL_params[f"dB{(no_of_layers+1)}"]=np.sum(dL_params[f"dW{(no_of_layers+1)}"].T,axis=1,keepdims=True)
    ###for intermdeiate layer
    dZ = np.multiply(sigmoid_derivative(net["A2"]),np.dot(net["W3"],error))
    dL_params[f"dW{(no_of_layers)}"]=np.dot(net["A1"],dZ.T)
    dL_params[f"dB{(no_of_layers)}"]=np.sum(dL_params[f"dW{(no_of_layers)}"].T,axis=1,keepdims=True)
    ###for first layer
    dZ1=np.multiply(sigmoid_derivative(net["A1"]),np.dot(net["W2"],dZ))
    delta=np.dot(net["W1"],dZ1)
    dL_params[f"dW{(no_of_layers-1)}"]=np.dot(sample,dZ1.T)
    dL_params[f"dB{(no_of_layers-1)}"]=np.sum(dL_params[f"dW{(no_of_layers-1)}"].T,axis=1,keepdims=True)
    return dL_params[f"dW{(no_of_layers+1)}"],dL_params[f"dB{(no_of_layers+1)}"],dL_params[f"dW{(no_of_layers)}"],dL_params[f"dB{(no_of_layers)}"],dL_params[f"dW{(no_of_layers-1)}"],dL_params[f"dB{(no_of_layers-1)}"],delta


In [0]:
def Update_params(W3,B3,W2,B2,W1,B1):
    net['W3'] -= lr*W3 ###weights updation
    net['B3'] -= lr*B3###bias updation
    net['W2'] -= lr*W2 ###weights updation
    net['B2'] -= lr*B2###bias updation
    net['W1'] -= lr*W1 ###weights updation
    net['B1'] -= lr*B1###bias updation
    return
def accuracy(X,Y,bias_1,bias_2):
    count=0;
    for i in range(0,len(X)):
      conv_res_1=forward(X[i],kernal1)  #Perform Convolution
      conv_res_1=relu(conv_res_1+bias_1)  #Add bias , Peroform Activation
      conv_res_2=forward(conv_res_1,kernal2) #Perform second convolution
      conv_res_2=relu(conv_res_2+bias_2)  #Add bais, and perform activation
      pooled_result_=Maxpool(conv_res_2) ### step 4: perform Max pooling
      size=pooled_result.shape[0]*pooled_result.shape[1]*pooled_result.shape[2]
      flatt=pooled_result
      flatt=flatt.reshape(size,2)
      flatten=[]
      for flat in flatt:
        flatten.append(flat[0])
      flatten=np.expand_dims(np.array(flatten),axis=-1)     #step 6: Flatten the Result
      y_pred=feedforward(net,flatten,no_of_layers)  ### Step 8: Send to FC layyers
      if np.argmax(y_pred)==np.argmax(Y[i]):
        count+=1
    print("Accuracy = ",count/len(Y))
    return count/len(Y)
def relu(s):
    return np.maximum(s,0);

def relu_derivative(s):
    s[s<=0] = 0
    s[s>0] = 1
    return s


In [0]:
def forward(a,b):
  a=a.T
  res=[]
  for each in a:
    res.append(convolution(each,b))
    rs=np.array(res)
  conv_res=np.sum(rs,axis=0)
  #print(conv_res.shape)
  return conv_res

In [14]:
print("Kernal 1 shape",kernal1.shape)
print("Kernal 2 shape",kernal2.shape)
conv_res_1=forward(x_train[0],kernal1)
bias_1=np.random.randn(conv_res_1.shape[0],conv_res_1.shape[1],conv_res_1.shape[2])
print("Conv 1 Result:",conv_res_1.shape)
print("Bias 1 Shape:",conv_res_1.shape)
conv_res_2=forward(conv_res_1,kernal2)
bias_2=np.random.randn(conv_res_2.shape[0],conv_res_2.shape[1],conv_res_2.shape[2])
print("Conv 2 Result:",conv_res_2.shape)
print("Bias 2 Result:",bias_2.shape)
pooled_result=Maxpool(conv_res_2)
print("Pooled Result Shape",pooled_result.shape)
no_of_layers=2;
input_dim=pooled_result.shape[0]*pooled_result.shape[1]*pooled_result.shape[2]
neurons_per_layer=[512,256,10]
net=init_network(no_of_layers,input_dim,neurons_per_layer)

Kernal 1 shape (5, 5, 18)
Kernal 2 shape (3, 3, 18)
Conv 1 Result: (28, 28, 18)
Bias 1 Shape: (28, 28, 18)
Conv 2 Result: (26, 26, 18)
Bias 2 Result: (26, 26, 18)
Pooled Result Shape (18, 13, 13, 2)
W1 Shape (3042, 512)
B1 Shape (512, 1)
Z1 Shape (512, 1)
A1 Shape (512, 1)
W2 Shape (512, 256)
B2 Shape (256, 1)
Z2 Shape (256, 1)
A2 Shape (256, 1)
W3 Shape (256, 10)
B3 Shape (10, 1)
Z3 Shape (10, 1)
A3 Shape (10, 1)


In [15]:
print(bias_1.shape)
print(bias_2.shape)

(28, 28, 18)
(26, 26, 18)


In [16]:
indices=[]
for i in range(0,len(x_train),5000):
    indices.append(random.sample(range(i, i+5000-1),500))
a=[]
b=[]
for index in indices:
    a.append(x_train[index])
    b.append(y_train[index])
X=np.array(a)
Y=np.array(b)
X=np.concatenate((X[0],X[1],X[2],X[3],X[4],X[5],X[6],X[7],X[8],X[9]),axis=0)
Y=np.concatenate((Y[0],Y[1],Y[2],Y[3],Y[4],Y[5],Y[6],Y[7],Y[8],Y[9]),axis=0)
print(X.shape)
print(Y.shape)

(5000, 32, 32, 3)
(5000, 10)


In [0]:
accu_vect=[]
loss_vect=[]
lr=0.001
epochs=20

In [0]:
for j in range(0,epochs):
    print("Epoch #",j+1)
    loss=0
    p=np.random.permutation(len(X))
    trX=X[p]
    trY=Y[p]
    for i in range(0,len(trX)):
        print("Image #",i+1)
        conv_res_1=conv_forward(trX[i],kernal1)  #Perform Convolution
        conv_res_1=relu(conv_res_1+bias_1)  #Add bias , Peroform Activation
        conv_res_2=conv_forward(conv_res_1,kernal2) #Perform second convolution
        conv_res_2=relu(conv_res_2+bias_2)  #Add bais, and perform activation
        pooled_result_=Maxpool(conv_res_2) ### step 4: perform Max pooling
        maxID=Mask(pooled_result,conv_res_2)  #Step 5: Find Mask while in forward pass
        #print("Mask Shape",maxID.shape)
        size=pooled_result.shape[0]*pooled_result.shape[1]*pooled_result.shape[2]
        flatt=pooled_result
        flatt=flatt.reshape(size,2)
        flatten=[]
        for flat in flatt:
            flatten.append(flat[0])
        flatten=np.expand_dims(np.array(flatten),axis=-1)     #step 6: Flatten the Result
        y_pred=feedforward(net,flatten,no_of_layers)  ### Step 8: Send to FC layyers
        loss+=crossentropyloss(y_pred,trY[i]) #step 9: Compute loss
        W3,B3,W2,B2,W1,B1,dL=Backwardpropagate(flatten,y_pred,trY[i],net,lr)#Step 10: Backprop of FC layer(s)
        dL=dL.reshape(pooled_result.shape[1],pooled_result.shape[2],pooled_result.shape[0]) #Step 11: Unflatten the flattend
        dL=np.concatenate((dL,dL),axis=0)
        dL=np.concatenate((dL,dL),axis=1)
        #print("dL Shape",dL.shape)
        dLmaxID=np.multiply(dL,maxID)   #Multiply Mask with dL. Backprop of Maxpoooling Layer
        dLmaxID=relu_derivative(dLmaxID)
        dL_b2= dLmaxID          ####used to update b1
        dLmaxID=np.fliplr(dLmaxID) #flip horizontally
        dLmaxID=np.flipud(dLmaxID) #flip vertically
        ######Max-Pooling Backprop Done####################
        stride=1
        #pad=int((stride*(conv_res_1.shape[0]-1)-(dLmaxID.shape[0]-kernal2.shape[1]))/4)
        #print(pad)
        pad=2
        dL2=np.pad(dLmaxID,pad,mode='constant')[:,:,pad:-pad]
        #print("dL2 shape",dL2.shape)
        #print("dL2 after padding",dL2.shape)
        k2=conv_forward(conv_res_1,dLmaxID)   ####filter 2 update: conv(input to previous layer,rotated Loss)
        #print("K2 loss shape",k2.shape)
        #################Filter for second conv layer Updated############
        # pad=int((stride*(trX[i].shape[0]-1)-(dL2.shape[0]-kernal1.shape[1]))/4)
        k2_flip=np.fliplr(kernal2)
        k2_flip=np.flipud(k2_flip)
        dL1=conv_forward(dL2,k2_flip)
        dL1=relu_derivative(dL1)
        dL_b1=dL1  #####Used to update Bias
        #print("dL1 shape",dL1.shape)
        # dL1=np.pad(dL1,pad,mode='constant')[:,:,pad:-pad]
        # print("dL1 after padding",dL1.shape)
        k1=conv_forward(trX[i],dL1) ### filter 1 update
        #print("K1 loss shape",k1.shape)

        #################Filter for second conv layer Updated############
        #print("K1 loss shape",k1.shape)
        Update_params(W3,B3,W2,B2,W1,B1)
        kernal1-=lr*k1
        kernal2-=lr*k2
        bias_1-=lr*dL_b1
        bias_2-=dLmaxID
        if(i%1000==0 and i>0):
          q=random.sample(range(0,i),300)
          valX=trX[q]
          valY=trY[q]
          print("After",i,"Images")
          accu_vect.append(accuracy(valX,valY,bias_1,bias_2))
          print("Loss = ",loss/i)
    loss_vect.append(loss)

Epoch # 1
Image # 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


Image # 2
Image # 3
Image # 4
Image # 5
Image # 6
Image # 7
Image # 8
Image # 9
Image # 10
Image # 11
Image # 12
Image # 13
Image # 14
Image # 15
Image # 16
Image # 17
Image # 18
Image # 19
Image # 20
Image # 21
Image # 22
Image # 23
Image # 24
Image # 25
Image # 26
Image # 27
Image # 28
Image # 29
Image # 30
Image # 31
Image # 32
Image # 33
Image # 34
Image # 35
Image # 36
Image # 37
Image # 38
Image # 39
Image # 40
Image # 41
Image # 42
Image # 43
Image # 44
Image # 45
Image # 46
Image # 47
Image # 48
Image # 49
Image # 50
Image # 51
Image # 52
Image # 53
Image # 54
Image # 55
Image # 56
Image # 57
Image # 58
Image # 59
Image # 60
Image # 61
Image # 62
Image # 63
Image # 64
Image # 65
Image # 66
Image # 67
Image # 68
Image # 69
Image # 70
Image # 71
Image # 72
Image # 73
Image # 74
Image # 75
Image # 76
Image # 77
Image # 78
Image # 79
Image # 80
Image # 81
Image # 82
Image # 83
Image # 84
Image # 85
Image # 86
Image # 87
Image # 88
Image # 89
Image # 90
Image # 91
Image # 92
Image #